In [156]:
# import moduel
import pandas as pd
import numpy as np
import os 
from os import listdir
import re

base_dir = os.getcwd() # 현재 작업 dir 확인

# 데이터가 들어있는 폴더
data_dir = "C:\\Users\\User\\Documents\\data_mise\\{}".format('2017')
# data_dir로 작업하는 폴더 경로 변경
working_dir = os.chdir(data_dir) 

# 전처리된 데이터를 내보낼 폴더
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\{}\\".format('2017_전처리') 
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
        os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.

# 전처리할 파일들이 들어있는 폴더의 파일 목록을 list형태로 추출
files = os.listdir(data_dir)
# 최종적으로 합쳐질 빈 데이터 프레임 생성
Merged_DF = pd.DataFrame(columns=['측정소코드', '측정소명', '측정일시', 
                                  'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25','주소'])

# for문을 돌면서 하나씩 파일을 꺼내옴
for file in files:
    raw_file = pd.read_csv(file, encoding='cp949')
    raw_file = raw_file.rename(columns={'Unnamed: 0': '지역'})
    raw_file = raw_file.loc[:, ~raw_file.columns.str.contains('^Unnamed')] # 정규표현식 활용해서 Unnamed columns 삭제

    # 전부 결측치인 column인 co2, vocs 두개의 열 제거
#         raw_file = raw_file.drop(['co2','vocs'], axis='columns')
    raw_file = raw_file.drop(['주소'], axis='columns')

    # 분석에서 유의미하게 사용되지 않을 실외측정기 구분 열 제거
#         raw_file = raw_file.drop(['flag'], axis='columns')

    # 데이터 결측치 처리 code -> Not a Number로 변경
    raw_file = raw_file.replace([-999,-9999],[np.nan, np.nan])

    # 데이터 이상치 처리 code -> 바로 앞의 값으로 대체하는 논리구조
#     pm10의 이상치 제거
    raw_file[raw_file['PM10'] > 1050.0].index
    outlier = raw_file[raw_file['PM10'] > 1050.0].index
    raw_file['PM10'].loc[outlier] = raw_file['PM10'].loc[outlier-1]

    raw_file = raw_file[raw_file['지역'].str.contains('서울')]
#         # pm25의 이상치 제거
#         raw_file[raw_file['pm25'] > 3800.0].index
#         outlier = raw_file[raw_file['pm25'] > 3800.0].index
#         raw_file['pm25'].loc[outlier] = raw_file['pm25'].loc[outlier-1]

#         # column 이름 변경
#         raw_file.rename(columns={
#             'serial': '측정기 고유번호',
#     #         'flag': '실외 측정기 구분',
#             'pm10': '미세먼지(PM-10)',
#     #         'co2': '이산화탄소 농도(ppm)',
#     #         'vocs': '휘발성유기화합물 농도',
#             'noise': '소음(db)',
#             'temp': '온도(℃)',
#             'humi': '습도(%)',
#             'pm25': '초미세먼지(PM-2.5)',
#         }, inplace=True)

    # datetime형식으로 type 변경
    # 이 방식은 반드시 마지막에 해주어야 오류가 발생하지 않는다.
    raw_file['측정일시'] = raw_file['측정일시'].astype('str')
    date_ls = list(raw_file['측정일시']) # 날짜 데이터를 list형태로 변경

    # 시간별로 date를 절사. :13까지가 시간단위!
    days_ls = []
    for date in date_ls:
        date = date[:8]
#         date = date.replace(' ','-')
#         date = date.replace('-','')
        days_ls.append(date)
    raw_file['측정일시'] = days_ls

    # 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
    raw_file['측정일시'] = pd.to_datetime(raw_file['측정일시'], format='%Y%m%d')
#     raw_file['측정일시'] = raw_file['측정일시'].astype('str')
    
#     raw_file['측정일시'] = pd.to_datetime(raw_file['측정일시']).dt.date
    
#     raw_file['측정일시'] = pd.to_datetime(raw_file['측정일시'], format='%Y%m%d')

#     print(raw_file.columns) # columns name 확인
#     print(raw_file.describe())
#     print(raw_file['미세먼지(PM-10)'].max())
#     print(raw_file['초미세먼지(PM-2.5)'].max())
#     print(raw_file.tail())

    # 바뀐 dataframe을 새로운 폴더의 같은 이름으로 to_csv 하는 code
#         raw_file.to_csv(out_dir+file)
#         print(file,"작업완료")

#     raw_file = raw_file.drop_duplicates(['지역'], keep='last')

    # 위에서 생성한 빈 데이터 프레임에 연결
    Merged_DF = pd.concat([Merged_DF, raw_file])
    Merged_DF = Merged_DF.reset_index(drop=True)

    print(raw_file.columns)
    print(len(raw_file))
    print(raw_file['PM10'].max())
    print(raw_file['PM25'].min())
    raw_file.head()
# 합쳐진 df을 다른 이름으로 빼내기
#     Merged_DF.to_csv(out_dir+'Total_{}.csv'.format(DataDirectoryName))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Index(['지역', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
84240
293.0
1.0
Index(['지역', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
85176
423.0
1.0
Index(['지역', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
86112
182.0
3.0
Index(['지역', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
86112
242.0
3.0


In [157]:
Merged_DF['지역'].value_counts()

서울 성동구     17520
서울 종로구     17520
서울 동대문구    17520
서울 마포구     17520
서울 강남구     17520
서울 동작구     17520
서울 성북구     17520
서울 강동구     17520
서울 용산구     17520
서울 노원구     17520
서울 서초구     17520
서울 영등포구    17520
서울 관악구      8760
서울 도봉구      8760
서울 금천구      8760
서울 중구       8760
서울 강북구      8760
서울 중구 청     8760
서울 광진구      8760
서울 강서구      8760
서울 강서로      8760
서울 구로구      8760
서울 은평구      8760
서울 양천구      8760
서울 중랑구      8760
서울 서대문구     8760
서울 송파구      8760
Name: 지역, dtype: int64

In [ ]:
# Merged_DF = Merged_DF.sort_values(by='지역')
# Merged_DF = Merged_DF.reset_index(drop=True)
# Merged_DF

In [163]:
test = round(Merged_DF.groupby(['측정일시', '지역']).mean(),2)
test.head()

CO   NO2    O3   PM10   PM25   SO2
측정일시       지역                                           
2017-01-01 서울 강남구   0.98  0.04  0.00  78.56  57.79  0.01
           서울 강동구   0.88  0.04  0.00  81.25  54.21  0.00
           서울 강북구   0.98  0.05  0.00  78.17  65.21  0.00
           서울 강서구   0.90  0.06  0.00  84.08    NaN  0.01
           서울 강서로   0.96  0.05  0.01  75.12  53.64  0.01

In [164]:
len(test)

9855

In [166]:
test.to_csv('test.csv', encoding='cp949')

PermissionError: [Errno 13] Permission denied: 'test.csv'

## 마무리

In [123]:
data_dir = "C:\\Users\\User\\Documents\\data_mise\\{}".format('2017')
# data_dir로 작업하는 폴더 경로 변경
working_dir = os.chdir(data_dir) 

df = pd.read_csv('2017_total.csv',encoding='cp949')
df

,지역,측정일시,CO,NO2,O3,PM10,PM25,SO2
0,서울 강남구,2017010101,1.00,0.0410,0.0020,62.5,48.0,0.0050
1,서울 강남구,2017010102,1.00,0.0400,0.0020,63.0,44.0,0.0050
2,서울 강남구,2017010103,1.10,0.0390,0.0020,71.0,44.0,0.0045
3,서울 강남구,2017010104,1.00,0.0360,0.0020,79.5,44.0,0.0045
4,서울 강남구,2017010105,0.90,0.0325,0.0020,71.5,40.0,0.0045
5,서울 강남구,2017010106,1.00,0.0350,0.0020,60.5,51.0,0.0050
6,서울 강남구,2017010107,0.90,0.0325,0.0020,63.5,56.0,0.0045
7,서울 강남구,2017010108,0.90,0.0325,0.0015,56.5,50.0,0.0045
8,서울 강남구,2017010109,1.00,0.0340,0.0020,58.0,44.0,0.0045
9,서울 강남구,2017010110,0.95,0.0330,0.0020,55.0,48.0,0.0045


In [125]:
df = round(df.groupby('지역').mean(),2)
df = df.drop(columns='측정일시')
df

,CO,NO2,O3,PM10,PM25,SO2
지역,,,,,,
서울 강남구,0.57,0.03,0.02,48.44,24.99,0.01
서울 강동구,0.52,0.04,0.02,49.00,23.79,0.00
서울 강북구,0.47,0.02,0.03,37.01,21.83,0.00
서울 강서구,0.52,0.05,0.02,65.40,NaN,0.01
서울 강서로,0.48,0.03,0.03,47.01,23.68,0.01
서울 관악구,0.50,0.03,0.02,43.19,26.07,0.01
서울 광진구,0.57,0.03,0.03,41.54,24.75,0.00
서울 구로구,0.52,0.02,0.02,45.30,24.96,0.01
서울 금천구,0.51,0.03,0.03,41.32,25.72,0.00


In [127]:
df.to_csv('2017_final.csv', encoding='cp949')

## 2018

In [167]:
# import moduel
import pandas as pd
import numpy as np
import os 
from os import listdir
import re

# # DataDirectoryName: 데이터가 들어있는 폴더 이름을 입력하세요
# # OutDirectoryName: 전처리된 데이터를 내보낼 폴더 이름을 입력하세요
# def PreprocessingData(DataDirectoryName, OutDirectoryName):
base_dir = os.getcwd() # 현재 작업 dir 확인

# 데이터가 들어있는 폴더
data_dir = "C:\\Users\\User\\Documents\\data_mise\\{}".format('2018')
# data_dir로 작업하는 폴더 경로 변경
working_dir = os.chdir(data_dir) 

# 전처리된 데이터를 내보낼 폴더
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\{}\\".format('2018_전처리') 
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
        os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.

# 전처리할 파일들이 들어있는 폴더의 파일 목록을 list형태로 추출
files = os.listdir(data_dir)
# 최종적으로 합쳐질 빈 데이터 프레임 생성
Merged_DF = pd.DataFrame(columns=['측정소코드', '측정소명', '측정일시', 
                                  'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25','주소'])

# for문을 돌면서 하나씩 파일을 꺼내옴
for file in files:
    raw_file = pd.read_csv(file, encoding='cp949')
    raw_file = raw_file.rename(columns={'Unnamed: 0': '지역'})
    raw_file = raw_file.loc[:, ~raw_file.columns.str.contains('^Unnamed')] # 정규표현식 활용해서 Unnamed columns 삭제

    # 전부 결측치인 column인 co2, vocs 두개의 열 제거
#         raw_file = raw_file.drop(['co2','vocs'], axis='columns')
    raw_file = raw_file.drop(['주소'], axis='columns')

    # 분석에서 유의미하게 사용되지 않을 실외측정기 구분 열 제거
#         raw_file = raw_file.drop(['flag'], axis='columns')

    # 데이터 결측치 처리 code -> Not a Number로 변경
    raw_file = raw_file.replace([-999,-9999],[np.nan, np.nan])

    # 데이터 이상치 처리 code -> 바로 앞의 값으로 대체하는 논리구조
#     pm10의 이상치 제거
    raw_file[raw_file['PM10'] > 1050.0].index
    outlier = raw_file[raw_file['PM10'] > 1050.0].index
    raw_file['PM10'].loc[outlier] = raw_file['PM10'].loc[outlier-1]

    raw_file = raw_file[raw_file['지역'].str.contains('서울')]
#         # pm25의 이상치 제거
#         raw_file[raw_file['pm25'] > 3800.0].index
#         outlier = raw_file[raw_file['pm25'] > 3800.0].index
#         raw_file['pm25'].loc[outlier] = raw_file['pm25'].loc[outlier-1]

#         # column 이름 변경
#         raw_file.rename(columns={
#             'serial': '측정기 고유번호',
#     #         'flag': '실외 측정기 구분',
#             'pm10': '미세먼지(PM-10)',
#     #         'co2': '이산화탄소 농도(ppm)',
#     #         'vocs': '휘발성유기화합물 농도',
#             'noise': '소음(db)',
#             'temp': '온도(℃)',
#             'humi': '습도(%)',
#             'pm25': '초미세먼지(PM-2.5)',
#         }, inplace=True)

    # datetime형식으로 type 변경
    # 이 방식은 반드시 마지막에 해주어야 오류가 발생하지 않는다.
    raw_file['측정일시'] = raw_file['측정일시'].astype('str')
    date_ls = list(raw_file['측정일시']) # 날짜 데이터를 list형태로 변경

    # 시간별로 date를 절사. :13까지가 시간단위!
    days_ls = []
    for date in date_ls:
        date = date[:8]
#         date = date.replace(' ','-')
#         date = date.replace('-','')
        days_ls.append(date)
    raw_file['측정일시'] = days_ls

    # 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
    raw_file['측정일시'] = pd.to_datetime(raw_file['측정일시'], format='%Y%m%d')
#     raw_file['측정일시'] = raw_file['측정일시'].astype('str')
    
#     raw_file['측정일시'] = pd.to_datetime(raw_file['측정일시']).dt.date
    
#     raw_file['측정일시'] = pd.to_datetime(raw_file['측정일시'], format='%Y%m%d')

#     print(raw_file.columns) # columns name 확인
#     print(raw_file.describe())
#     print(raw_file['미세먼지(PM-10)'].max())
#     print(raw_file['초미세먼지(PM-2.5)'].max())
#     print(raw_file.tail())

    # 바뀐 dataframe을 새로운 폴더의 같은 이름으로 to_csv 하는 code
#         raw_file.to_csv(out_dir+file)
#         print(file,"작업완료")

#     raw_file = raw_file.drop_duplicates(['지역'], keep='last')

    # 위에서 생성한 빈 데이터 프레임에 연결
    Merged_DF = pd.concat([Merged_DF, raw_file])
    Merged_DF = Merged_DF.reset_index(drop=True)

    print(raw_file.columns)
    print(len(raw_file))
    print(raw_file['PM10'].max())
    print(raw_file['PM25'].min())
    raw_file.head()
# 합쳐진 df을 다른 이름으로 빼내기
#     Merged_DF.to_csv(out_dir+'Total_{}.csv'.format(DataDirectoryName))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Index(['지역', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
84240
330.0
1.0
Index(['지역', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
85176
470.0
1.0
Index(['지역', '망', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
86112
105.0
1.0
Index(['지역', '망', '측정소코드', '측정소명', '측정일시', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')
87576
405.0
1.0


In [168]:
test = Merged_DF.groupby(['측정일시', '지역']).mean()
test.head()

CO       NO2        O3       PM10       PM25  \
측정일시       지역                                                            
2018-01-01 서울 강남구   0.639583  0.036646  0.008896  43.250000  22.333333   
           서울 강동구   0.654167  0.042167  0.008438  48.458333  24.391304   
           서울 강북구   0.612500  0.026083  0.018208  38.458333  18.291667   
           서울 강서구   0.641667  0.045458  0.006875  50.583333        NaN   
           서울 강서로   0.695833  0.037750  0.011208        NaN  13.416667   

                         SO2  
측정일시       지역                 
2018-01-01 서울 강남구   0.006063  
           서울 강동구   0.005292  
           서울 강북구   0.004333  
           서울 강서구   0.006292  
           서울 강서로   0.004375

In [169]:
test.to_csv('test.csv', encoding='cp949')

## 마무리

In [119]:
df = pd.read_csv('2018_total.csv',encoding='cp949')
df

,지역,측정일시,CO,NO2,O3,PM10,PM25,SO2
0,서울 강남구,2018010101,0.55,0.0285,0.0110,36.0,17.0,0.0050
1,서울 강남구,2018010102,0.55,0.0265,0.0120,37.0,20.0,0.0050
2,서울 강남구,2018010103,0.50,0.0265,0.0115,34.0,14.0,0.0055
3,서울 강남구,2018010104,0.55,0.0315,0.0085,31.5,17.0,0.0055
4,서울 강남구,2018010105,0.70,0.0375,0.0055,36.0,17.0,0.0050
5,서울 강남구,2018010106,0.75,0.0410,0.0035,38.0,25.0,0.0055
6,서울 강남구,2018010107,0.75,0.0400,0.0035,38.0,21.0,0.0055
7,서울 강남구,2018010108,0.75,0.0440,0.0025,35.5,19.0,0.0060
8,서울 강남구,2018010109,0.75,0.0455,0.0030,34.0,21.0,0.0050
9,서울 강남구,2018010110,0.75,0.0425,0.0045,38.0,25.0,0.0055


In [120]:
df = round(df.groupby('지역').mean(),2)
df = df.drop(columns='측정일시')
df

,CO,NO2,O3,PM10,PM25,SO2
지역,,,,,,
서울 강남구,0.59,0.03,0.02,41.19,23.66,0.00
서울 강동구,0.53,0.04,0.02,44.18,23.18,0.00
서울 강북구,0.44,0.02,0.03,35.37,19.11,0.00
서울 강서구,0.54,0.04,0.02,49.03,25.92,0.01
서울 강서로,0.47,0.03,0.02,38.75,22.20,0.00
서울 관악구,0.43,0.03,0.02,43.46,27.19,0.00
서울 광진구,0.59,0.03,0.02,39.57,20.91,0.00
서울 구로구,0.35,0.03,0.03,42.69,22.75,0.00
서울 금천구,0.81,0.05,0.01,44.74,24.19,0.00


In [121]:
df.to_csv('2018_final.csv', encoding='cp949')

In [196]:
data_dir = "C:\\Users\\User\\Documents\\data_mise\\"
# data_dir로 작업하는 폴더 경로 변경
working_dir = os.chdir(data_dir) 

_2017 = pd.read_csv('2017_total.csv',encoding='cp949')
_2018 = pd.read_csv('2018_total.csv',encoding='cp949')

_2017['지역'] = _2017['지역'].str.replace('서울 ', '')
_2018['지역'] = _2018['지역'].str.replace('서울 ', '')

M = _2017.append(_2018)
M = M.reset_index(drop=True)
M = M.rename(columns={'측정일시':'date'})
M

,date,지역,CO,NO2,O3,PM10,PM25,SO2
0,2017-01-01,강남구,0.977083,0.041479,0.002333,78.562500,57.791667,0.005021
1,2017-01-01,강동구,0.883333,0.043562,0.002792,81.250000,54.208333,0.004292
2,2017-01-01,강북구,0.984211,0.046375,0.002625,78.166667,65.208333,0.004875
3,2017-01-01,강서구,0.900000,0.063375,0.002783,84.083333,NaN,0.006542
4,2017-01-01,강서로,0.962500,0.053292,0.005250,75.125000,53.636364,0.006292
5,2017-01-01,관악구,1.258333,0.064333,0.005375,74.958333,55.750000,0.006375
6,2017-01-01,광진구,0.812500,0.044542,0.005042,79.708333,71.333333,0.004375
7,2017-01-01,구로구,0.983333,0.036333,0.006208,79.000000,54.375000,0.007000
8,2017-01-01,금천구,1.029167,0.053458,0.006083,65.958333,45.000000,0.004875
9,2017-01-01,노원구,1.114583,0.059257,0.004146,82.062500,65.208333,0.006271


In [197]:
M = M[M['지역'] != '금천구']

In [198]:
M['지역'].value_counts()

용산구     730
강동구     730
송파구     730
강서구     730
중구      730
종로구     730
강북구     730
마포구     730
성동구     730
금천구     730
도봉구     730
노원구     730
동작구     730
중랑구     730
동대문구    730
양천구     730
은평구     730
서초구     730
관악구     730
영등포구    730
강남구     730
광진구     730
구로구     730
서대문구    730
성북구     730
강서로     730
Name: 지역, dtype: int64

In [199]:
len(M)

18980

In [201]:
M.to_csv('2017X2018_서울미세먼지_합칠용.csv', encoding='cp949')

In [174]:
19041 - 61

18980

In [175]:
18980 - 18250

730

In [203]:
why = pd.read_csv('2017X2018_서울미세먼지_합칠용.csv', encoding='cp949', index_col = 0)
why

,date,지역,CO,NO2,O3,PM10,PM25,SO2
0,2017-01-01,강남구,0.977083,0.041479,0.002333,78.562500,57.791667,0.005021
1,2017-01-01,강동구,0.883333,0.043562,0.002792,81.250000,54.208333,0.004292
2,2017-01-01,강북구,0.984211,0.046375,0.002625,78.166667,65.208333,0.004875
3,2017-01-01,강서구,0.900000,0.063375,0.002783,84.083333,NaN,0.006542
5,2017-01-01,관악구,1.258333,0.064333,0.005375,74.958333,55.750000,0.006375
6,2017-01-01,광진구,0.812500,0.044542,0.005042,79.708333,71.333333,0.004375
7,2017-01-01,구로구,0.983333,0.036333,0.006208,79.000000,54.375000,0.007000
8,2017-01-01,금천구,1.029167,0.053458,0.006083,65.958333,45.000000,0.004875
9,2017-01-01,노원구,1.114583,0.059257,0.004146,82.062500,65.208333,0.006271
10,2017-01-01,도봉구,0.913043,0.052130,0.003087,78.304348,47.086957,0.005652


In [204]:
why['지역'].value_counts()

용산구     730
중랑구     730
강서구     730
중구      730
종로구     730
강북구     730
마포구     730
성동구     730
금천구     730
도봉구     730
노원구     730
동작구     730
동대문구    730
강동구     730
양천구     730
은평구     730
서초구     730
관악구     730
영등포구    730
강남구     730
광진구     730
구로구     730
서대문구    730
성북구     730
송파구     730
Name: 지역, dtype: int64

In [214]:
data_dir = "C:\\Users\\User\\Documents\\data_mise\\"
# data_dir로 작업하는 폴더 경로 변경
working_dir = os.chdir(data_dir) 

df = pd.read_csv('flow_park_mise_cp949.csv', encoding='cp949')
df['1인당면적라벨'] = np.nan
df['PM25단계'] = np.nan

good = df[df['1인당면적'] <= 2.0].index
df['1인당면적라벨'].loc[good] = '1'
soso = df[df['1인당면적'] > 2.0].index
df['1인당면적라벨'].loc[soso] = '2'
bad = df[df['1인당면적'] >= 3.0].index
df['1인당면적라벨'].loc[bad] = '3'
terrible = df[df['1인당면적'] >= 6.0].index
df['1인당면적라벨'].loc[terrible] = '4'

# PM 2.5
good = df[df['PM25'] < 16.0].index
df['PM25단계'].loc[good] = '좋음'
soso = df[df['PM25'] >= 16.0].index
df['PM25단계'].loc[soso] = '보통'
bad = df[df['PM25'] >= 36.0].index
df['PM25단계'].loc[bad] = '나쁨'
terrible = df[df['PM25'] >= 76.0].index
df['PM25단계'].loc[terrible] = '매우 나쁨'
# 1인당 면적
# 2<= 낮음
# 3 보통
# 3> 6<= 높음
# 6> 매우높음
df

,date,지역,총생활인구수,공원면적,공원수,미집행면적,예상총면적,1인당면적,공원당인구수,CO,O3,PM10,PM25,SO2,1인당면적라벨,PM25단계
0,2017-01-01,강남구,668041.0,2146595.0,134,805207.0,2951802.0,3.0,4985.0,0.977083,0.002333,78.562500,57.791667,0.005021,3,나쁨
1,2017-01-01,강동구,505314.0,1934993.0,74,898745.0,2833738.0,4.0,6829.0,0.883333,0.002792,81.250000,54.208333,0.004292,3,나쁨
2,2017-01-01,강북구,332492.0,1567126.0,45,468932.0,2036058.0,5.0,7389.0,0.984211,0.002625,78.166667,65.208333,0.004875,3,나쁨
3,2017-01-01,강서구,563938.0,3209226.0,139,1472431.0,4681657.0,6.0,4057.0,0.900000,0.002783,84.083333,NaN,0.006542,4,NaN
4,2017-01-01,관악구,529131.0,442644.0,76,289624.0,732268.0,1.0,6962.0,1.258333,0.005375,74.958333,55.750000,0.006375,1,나쁨
5,2017-01-01,광진구,385950.0,43788.0,38,12634.0,56422.0,0.0,10157.0,0.812500,0.005042,79.708333,71.333333,0.004375,1,나쁨
6,2017-01-01,구로구,408474.0,2705529.0,47,313974.0,3019503.0,7.0,8691.0,0.983333,0.006208,79.000000,54.375000,0.007000,4,나쁨
7,2017-01-01,금천구,206853.0,147893.0,51,NaN,NaN,1.0,4056.0,1.029167,0.006083,65.958333,45.000000,0.004875,1,나쁨
8,2017-01-01,노원구,563873.0,14457777.0,118,852658.0,15310435.0,26.0,4779.0,1.114583,0.004146,82.062500,65.208333,0.006271,4,나쁨
9,2017-01-01,도봉구,323466.0,1040386.0,6,820064.0,1860450.0,3.0,53911.0,0.913043,0.003087,78.304348,47.086957,0.005652,3,나쁨


In [215]:
df.to_csv('plz.csv', encoding='cp949')